# Introduction to DSGE.jl

Pearl Li <br> CEF 2017 <br> June 27, 2017

## Outline

1. Notation
2. Solving, estimating, and forecasting an existing model
3. Model implementation

## Notation

- $y_t$ is a vector of observables at time $t$
- $s_t$ is a vector of states, including expectations of future states and lags
- $\epsilon_t$ is a vector of exogenous shocks
- $\eta_t$ is a vector of rational expectations errors
- $\theta$ is a vector of parameters

Equilibrium conditions

$$\Gamma_0(\theta) s_t = \Gamma_1(\theta) s_{t-1} + \Psi(\theta) \epsilon_t + \Pi(\theta) \eta_t + C(\theta)$$

which are solved to give the state-space representation

$$
\begin{align*}
s_t &= T(\theta) s_{t-1} + R(\theta) \epsilon_t + C(\theta) \\
y_t &= Z(\theta) s_t + D(\theta)
\end{align*}
$$

## Using Existing Models

Let's construct an instance of the following (log-linearized) three-equation New Keynesian model:

$$
\begin{align*}
y_t - g_t &= -\frac{1}{\tau} R_t + \frac{1}{1 + \tau} \mathbb{E} \pi_{t+1} + \mathbb{E}(y_{t+1} - g_{t+1}) + \frac{1}{\tau} \mathbb{E} z_{t+1} \\
\pi_t &= \beta \mathbb{E} \pi_{t+1} + \kappa (y_t - g_t) \\
R_t &= \rho_R R_{t-1} + (1 - \rho_R) [\psi_1 \pi_t + \psi_2 (y_t - g_t)] + \epsilon_{R,t}
\end{align*}
$$

where

$$
\begin{align*}
s_t &= [y_t, \pi_t, R_t, y_{t-1}, g_t, z_t, \mathbb{E} y_{t+1}, \mathbb{E} \pi_{t+1}] \\
y_t &= [\text{Real per-capita GDP growth}_t, \text{CPI inflation}_t, \text{Nominal FFR}_t]
\end{align*}
$$

are the state and observable vectors respectively.

In [23]:
using DSGE

# Construct model object
m = AnSchorfheide()

# Set data vintage and initial forecast date
m <= Setting(:data_vintage, "170528")
m <= Setting(:date_forecast_start, DSGE.quartertodate("2017-Q2"))

# Set input and output directories
m <= Setting(:dataroot, joinpath(pwd(), "input_data"))
m <= Setting(:saveroot, pwd())

In [22]:
m

Dynamic Stochastic General Equilibrium Model
no. states:             8
no. anticipated shocks: 0
data vintage:           170528
description:
 Julia implementation of model defined in 'Bayesian Estimation of DSGE Models' by Sungbae An and Frank Schorfheide: AnSchorfheide, ss0


Estimate and forecast the model:

In [ ]:
# Find a posterior mode, sample from the posterior distribution
estimate(m)

# Add parallel workers
my_procs = addprocs(50)

# Forecast using the full distribution of parameters
output_vars = [:forecaststates, :forecastobs]
forecast_one(m, :full, :none, output_vars)
means_bands_all(m, :full, :none, output_vars)

# Remove parallel workers
rmprocs(my_procs)

Since in practice estimating and forecasting the full distribution is time-consuming, we'll read in an existing posterior mode...

In [2]:
# Read in existing mode
mode_file = rawpath(m, "estimate", "paramsmode.h5")
specify_mode!(m, mode_file)

Loaded previous mode from /Users/rcepxl10/Google Drive/frbny/cef-2017/CEF_2017_Workshop/dsgejl/output_data/an_schorfheide/ss0/estimate/raw/paramsmode_vint=170528.h5.


... and forecast using modal parameters:

In [9]:
# Load data
df = load_data(m)

# Forecast using modal parameters
output_vars = [:forecaststates, :forecastobs]
forecast_one(m, :mode, :none, output_vars, df = df, verbose = :none)
means_bands_all(m, :mode, :none, output_vars, verbose = :none)

Reading dataset /Users/rcepxl10/Google Drive/frbny/cef-2017/CEF_2017_Workshop/dsgejl/input_data/data/data_170528.csv from disk...dataset from disk valid


In [6]:
# Read in forecasted observables
files = get_meansbands_output_files(m, :mode, :none, output_vars)
mb = read_mb(files[:forecastobs])

MeansBands
  class: obs
  product: forecast
  cond: none
  para: mode
  dates: 2017-06-30 - 2032-03-31
  # of variables: 3
  bands: SubString{String}["50.0%"]


In [10]:
# Show first four forecasted quarters
mb.means[1:4, :]

,date,obs_gdp,obs_cpi,obs_nominalrate
1,2017-06-30,1.0247711232397094,11.315390469881926,1.3139454137230606
2,2017-09-30,1.1903805429393177,10.103585881441024,1.6682488733822125
3,2017-12-31,1.301650578197866,9.28739983248088,1.905189202533021
4,2018-03-31,1.3762101906441782,8.734358767470773,2.063046243299559


In [21]:
# Read in forecasted states
mb = read_mb(files[:forecaststates])

# Show first forecasted quarters of output, inflation, and interest rate
mb.means[1:4, [:y_t, :π_t, :R_t]]

,y_t,π_t,R_t
1,0.8118952136030042,0.2350315239936612,-0.23568746149574904
2,0.7161825375224499,0.16661955296739286,-0.14711159658096104
3,0.6486366042583442,0.12011639949265084,-0.0878765142932589
4,0.60014568887123,0.08840842601171174,-0.048412254101624384


## The Model Object

Subtypes of `AbstractModel` contain the following fields:

In [27]:
fieldnames(m)

16-element Array{Symbol,1}:
 :parameters                 
 :steady_state               
 :keys                       
 :endogenous_states          
 :exogenous_shocks           
 :expected_shocks            
 :equilibrium_conditions     
 :endogenous_states_augmented
 :observables                
 :spec                       
 :subspec                    
 :settings                   
 :test_settings              
 :rng                        
 :testing                    
 :observable_mappings        

These fields include:

- Vectors of time-invariant (`parameters`) and steady-state parameters (`steady_state`)
- `Dict{Symbol, Int}`s mapping states, shocks, expectational errors, equations, or observables to indices. For example:

In [28]:
m.endogenous_states

Dict{Symbol,Int64} with 8 entries:
  :y_t   => 1
  :g_t   => 5
  :π_t   => 2
  :Ey_t1 => 7
  :Eπ_t1 => 8
  :y_t1  => 4
  :R_t   => 3
  :z_t   => 6

`m.endogenous_states[:y_t] = 1` indicates that the first element of the state vector

$$s_t = [y_t, \pi_t, R_t, y_{t-1}, g_t, z_t, \mathbb{E} y_{t+1}, \mathbb{E} \pi_{t+1}]$$

is $y_t$.

In [29]:
m.equilibrium_conditions

Dict{Symbol,Int64} with 8 entries:
  :eq_phillips => 2
  :eq_mp       => 3
  :eq_Ey       => 7
  :eq_y_t1     => 4
  :eq_euler    => 1
  :eq_Eπ       => 8
  :eq_z        => 6
  :eq_g        => 5

`m.equilibrium_conditions[:eq_euler] = 1` means the first row of the equilibrium conditions

$$\Gamma_0 s_t = \Gamma_1 s_{t-1} + \Psi \epsilon_t + \Pi \eta_t + C$$

is the consumption Euler equation.

Additional fields in the model object:

- Strings giving the model specification (`spec`) and subspecification (`subspec`)
- `Dict{Symbol, Setting}`s of model settings, both for regular use (`settings`) and testing the package (`test_settings`)